In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
pd.set_option('display.max_columns', None)

In [60]:
file_path = "../data/homicidios.xlsx"
df = pd.read_excel(file_path, sheet_name="HECHOS")


La primera intervencion sera estandarizar la escritura de todos los valores del set de datos a minúsculas para obtener consistencia, facilidad en busquedas y comparaciones. Los encabezados de las columnas se modificaran para que esten escritos completamente en mayusculas.

In [61]:
# Encabezados en mayúsculas
df.columns = df.columns.str.upper()

# Seleccionar solo las columnas no numéricas
columnas_no_numericas = df.select_dtypes(exclude=['number']).columns

# Aplicar la conversión a minúsculas en esas columnas
df[columnas_no_numericas] = df[columnas_no_numericas].map(lambda x: x.lower() if isinstance(x, str) else x)


Realizaremos el proceso de limpieza teniendo en cuenta las situaciones más comunes:

1. Datos faltantes en algunas celdas
2. Columnas irrelevantes (que no responden al problema que queremos resolver)
3. Registros (filas) repetidos
4. Valores extremos (*outliers*) en el caso de las variables numéricas.

Al final de este proceso de limpieza deberíamos tener un set de datos **íntegro**, listo para la fase de Análisis Exploratorio.

In [62]:
print(df.shape)
df.head()

(696, 21)


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,ALTURA,CRUCE,DIRECCIÓN NORMALIZADA,COMUNA,XY (CABA),POS X,POS Y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,av piedra buena y av fernandez de la cruz,avenida,piedra buena av.,NaN,"fernandez de la cruz, f., gral. av.","piedra buena av. y fernandez de la cruz, f., g...",8,point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,moto-auto,moto,auto
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,av gral paz y av de los corrales,gral paz,"paz, gral. av.",NaN,de los corrales av.,"paz, gral. av. y de los corrales av.",9,point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,auto-pasajeros,auto,pasajeros
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,av entre rios 2034,avenida,entre rios av.,2034.0,NaN,entre rios av. 2034,1,point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,moto-auto,moto,auto
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,av larrazabal y gral villegas conrado,avenida,larrazabal av.,NaN,"villegas, conrado, gral.","larrazabal av. y villegas, conrado, gral.",8,point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,moto-sd,moto,sd
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,av san juan y presidente luis saenz peña,avenida,san juan av.,NaN,"saenz pe?a, luis, pres.","san juan av. y saenz peã‘a, luis, pres.",1,point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,moto-pasajeros,moto,pasajeros


In [63]:
duplicados_totales = df.duplicated().sum()
duplicados_totales

0

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  CALLE                  695 non-null    object        
 11  ALTURA                 129 non-null    float64       
 12  CRUCE                  525 non-null    object        
 13  DIREC

Los datos no están completos, pues no todas las columnas tienen la misma cantidad de registros.

El número total de registros debería ser 696. Sin embargo las columnas "Altura", "Cruce", "Dirección Normalizada" y "Calle" tienen datos faltantes.

Por ser un set de datos reducido, optaremos por imputar las filas correspondientes.

Revisaremos en primer lugar los registros faltantes en "Calle" y "Dirección Normalizada" para ver que ocurre en esos casos, considerando que es una cantidad pequeña.

In [65]:
# Filtrar registros donde la columna "Calle" es nula
calle_nula = df[df['CALLE'].isnull()]

# Filtrar registros donde la columna "Direccion normalizada" es nula
direccion_nula = df[df['DIRECCIÓN NORMALIZADA'].isnull()]

In [66]:
calle_nula

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,ALTURA,CRUCE,DIRECCIÓN NORMALIZADA,COMUNA,XY (CABA),POS X,POS Y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,sd,calle,NaN,NaN,NaN,NaN,0,point (. .),.,.,peaton-sd,peaton,sd


Al ser solo un registro que cuenta con varios campos sin especificar, se decide eliminarlo del dataset

In [67]:
df = df[df['ID'] != '2016-0151']

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  CALLE                  695 non-null    object        
 11  ALTURA                 129 non-null    float64       
 12  CRUCE                  525 non-null    object        
 13  DIRECCIÓN 

In [69]:
direccion_nula

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,ALTURA,CRUCE,DIRECCIÓN NORMALIZADA,COMUNA,XY (CABA),POS X,POS Y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,autopista lugones pk 10000,autopista,"lugones, leopoldo av.",NaN,NaN,NaN,13,point (. .),.,.,moto-sd,moto,sd
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,au buenos aires - la plata km. 4,autopista,autopista buenos aires - la plata,NaN,NaN,NaN,4,point (. .),.,.,moto-cargas,moto,cargas
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,sd,calle,NaN,NaN,NaN,NaN,0,point (. .),.,.,peaton-sd,peaton,sd
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,au perito moreno y ramal enlace au1/au6,autopista,autopista perito moreno,NaN,NaN,NaN,9,point (. .),.,.,moto-cargas,moto,cargas
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,au dellepiane 2400,autopista,autopista dellepiane luis tte. gral.,NaN,NaN,NaN,7,point (. .),.,.,auto-auto,auto,auto
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,autopista lugones km 4.7,autopista,"lugones, leopoldo av.",NaN,NaN,NaN,14,point (. .),.,.,peaton-auto,peaton,auto
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"lugones, leopoldo av. km 6,1",autopista,"lugones, leopoldo av.",NaN,NaN,NaN,14,point (. .),.,.,moto-objeto fijo,moto,objeto fijo
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,"au buenos aires la plata km 4,5",autopista,autopista buenos aires - la plata,NaN,NaN,NaN,4,point (. .),.,.,moto-cargas,moto,cargas


Notamos que todos los datos asociados a la ubicación no estan presentes en estos registros, por lo que se decide imputar como "sin datos", y tomar la columna Altura como un string ya que no tiene un sentido numérico continuo.

In [71]:
# Convertir la columna 'Altura' a tipo de dato string
# Convertir la columna 'Altura' a tipo de dato string y reemplazar NaN por 'None'
df['ALTURA'] = df['ALTURA'].astype(str).replace('nan', None)

# Imputación para Altura en registros con Dirección Normalizada nula
df.loc[df['DIRECCIÓN NORMALIZADA'].isnull(), 'ALTURA'] = 'sin datos'

# Imputación para Cruce en registros con Dirección Normalizada nula
df.loc[df['DIRECCIÓN NORMALIZADA'].isnull(), 'CRUCE'] = 'sin datos'

# Imputación para XY (CABA) en registros con Dirección Normalizada nula
df.loc[df['DIRECCIÓN NORMALIZADA'].isnull(), 'XY (CABA)'] = 'sin datos'

# Imputación para pos x y pos y en registros con Dirección Normalizada nula
df.loc[df['DIRECCIÓN NORMALIZADA'].isnull(), 'POS X'] = 'sin datos'
df.loc[df['DIRECCIÓN NORMALIZADA'].isnull(), 'POS Y'] = 'sin datos'

# Imputación para Dirección Normalizada
df['DIRECCIÓN NORMALIZADA'].fillna('sin datos', inplace=True)

Se verifica la ausencia de registros con datos asociados a la direccion nulos, ya que fueron imputados.

In [72]:
direccion_nula = df[df['DIRECCIÓN NORMALIZADA'].isnull()]
direccion_nula

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,ALTURA,CRUCE,DIRECCIÓN NORMALIZADA,COMUNA,XY (CABA),POS X,POS Y,PARTICIPANTES,VICTIMA,ACUSADO


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  CALLE                  695 non-null    object        
 11  ALTURA                 136 non-null    object        
 12  CRUCE                  532 non-null    object        
 13  DIRECCIÓN 

Ahora nos falta revisar un poco mas como tratar las columnas "Altura" y "Cruce". Al sumar las cantidades de no nulos en estas dos columnas (136 + 532 = 668) nos permite pensar que podria estar ocurriendo que para cuando una "Altura" no esta definida, es porque el hecho fue en un "Cruce" de calles y viceversa. Sin embargo revisaremos los valores donde no estan definidas "Cruce" y "Altura" simultaneamente.

In [74]:
cruce_altura_nulos = df.loc[(df['CRUCE'].isnull()) & (df['ALTURA'].isnull())]
cruce_altura_nulos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,ALTURA,CRUCE,DIRECCIÓN NORMALIZADA,COMUNA,XY (CABA),POS X,POS Y,PARTICIPANTES,VICTIMA,ACUSADO
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,autopista 1 sur presidente arturo frondizi km....,autopista,autopista 1 sur presidente arturo frondizi,None,NaN,autopista 1 sur presidente arturo frondizi,4,point (. .),-58.37714647568196,-34.63657525428238,sd-sd,sd,sd
64,2016-0087,1,2016-07-02,2016,7,3,00:10:00,0,autopista 1 sur pte arturo frondizi y av caseros,autopista,autopista 1 sur presidente arturo frondizi,None,NaN,autopista 1 sur presidente arturo frondizi y c...,1,point (107762.62066736 100018.90176187),-58.37864583,-34.62907067,moto-objeto fijo,moto,objeto fijo
71,2016-0096,1,2016-07-25,2016,7,25,07:00:00,7,"autopista dellepiane luis tte. gral. km. 2,3",autopista,autopista dellepiane luis tte. gral.,None,NaN,autopista dellepiane luis tte. gral.,8,point (. .),-58.47433193007387,-34.66684950051973,moto-cargas,moto,cargas
81,2016-0107,1,2016-08-20,2016,8,20,08:22:00,8,autopista 9 de julio sur alt av mendoza,autopista,autopista 1 sur presidente arturo frondizi,None,NaN,autopista 1 sur presidente arturo frondizi y d...,4,point (108408.31858686 97219.56218484),-58.37157668,-34.65429986,moto-auto,moto,auto
91,2016-0118,1,2016-09-04,2016,9,4,03:30:00,3,autopista 1 sur pte arturo frondizi km 2.9,autopista,autopista 1 sur presidente arturo frondizi,None,NaN,autopista 1 sur presidente arturo frondizi km....,1,point (107696.68171812 100254.76268710),-58.37936704,-34.62694503,auto-auto,auto,auto
100,2016-0130,1,2016-10-04,2016,10,4,12:30:00,12,autopista 9 de julio sur y av brasil,autopista,autopista 1 sur presidente arturo frondizi,None,NaN,autopista 1 sur presidente arturo frondizi y b...,1,point (107720.24059557 100176.85101454),-58.37910942,-34.62764717,moto-cargas,moto,cargas
127,2016-0160,1,2016-12-06,2016,12,6,05:30:00,5,autopista perito moreno altura velz,autopista,autopista perito moreno,None,NaN,autopista perito moreno (altura velez sarsfield),9,point (94867.91408532 99212.26216105),-58.51927194,-34.63635787,cargas-objeto fijo,cargas,objeto fijo
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,autopista 25 de mayo,autopista,autopista 25 de mayo,None,NaN,autopista 25 de mayo,0,point (. .),.,.,sd-sd,sd,sd
148,2017-0009,1,2017-01-16,2017,1,16,13:56:00,13,au 25 de mayo y av. boedo,autopista,autopista 25 de mayo,None,NaN,autopista 25 de mayo y boedo av.,5,point (104341.1368196 100202.74363294),-58.41595919,-34.62743346,moto-cargas,moto,cargas
155,2017-0016,1,2017-02-03,2017,2,3,05:12:21,5,"au 25 de mayo, kilometro 8.3, sentido hacia pr...",autopista,autopista 25 de mayo,None,NaN,autopista 25 de mayo km. 8.3,1,point (107547.76713701 100705.57738660),-58.38099494,-34.62288231,moto-cargas,moto,cargas


Observamos que los casos en los que la altura y la calle no estan definidas simultaneamente, pero en la mayoria de los casos tenemos informacion de la pos x y pos y por separado. Tambien hay algunos casos donde la posicion no esta definida en ninguna parte. Finalmente optamos por imputar todos los casos por "sin dato" ya que aun sin esa información especificada, los registros contienen otros valores que pueden ser utiles para otros analisis como el nombre y tipo de calle

In [75]:
df['CRUCE'].fillna('sin datos', inplace=True)
df['ALTURA'].fillna('sin datos', inplace=True)

Ya no tenemos nulos en nuestro set de datos

In [76]:
cruce_altura_nulos = df.loc[(df['CRUCE'].isnull()) & (df['ALTURA'].isnull())]
cruce_altura_nulos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,CALLE,ALTURA,CRUCE,DIRECCIÓN NORMALIZADA,COMUNA,XY (CABA),POS X,POS Y,PARTICIPANTES,VICTIMA,ACUSADO


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          695 non-null    object        
 10  CALLE                  695 non-null    object        
 11  ALTURA                 695 non-null    object        
 12  CRUCE                  695 non-null    object        
 13  DIRECCIÓN 

En la columna "Participantes" ocurre algo particular. Estan los datos de dos participantes separados por guión medio. Esto pudimos verlo en impresiones del dataframe *cruce_altura_nulos*. Para mayor comodidad y poder utilizar esta informacion mas facilmente en gráficos, separaremos estos datos en 2 columnas. "Participante A" y "Participante B"

In [78]:
# Dividir la columna 'Participantes' en dos nuevas columnas
df[['PARTICIPANTE A', 'PARTICIPANTE B']] = df['PARTICIPANTES'].str.split('-', n=1, expand=True)

# Eliminar la columna 'Participantes'
df.drop('PARTICIPANTES', axis=1, inplace=True)

In [79]:
df.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'CALLE', 'ALTURA', 'CRUCE',
       'DIRECCIÓN NORMALIZADA', 'COMUNA', 'XY (CABA)', 'POS X', 'POS Y',
       'VICTIMA', 'ACUSADO', 'PARTICIPANTE A', 'PARTICIPANTE B'],
      dtype='object')

Ya no tenemos nulos en nuestro set de datos y dividimos la columna "Participantes", pero aun queda estandarizar las imputaciones de valores, ya que por defecto, cuando no hay datos viene rellenado como "sd".

In [80]:
df.replace('sd', 'sin datos', inplace=True)

Se observó que la columna "LUGAR_DEL_HECHO" contiene una combinacion de la información de las columnas "Altura", "Cruce", "Calle" y en ocasiones alguna información extra no muy relevante que incluye numeros y "km". Se decide eliminar esta columna ya que para los fines del proyecto se desean los datos separados para poder evaluar relaciones y hacer interpretaciones.

In [81]:
df = df.drop('LUGAR_DEL_HECHO', axis=1)

Ahora revisaremos las 3 columnas dedicadas a posición, "XY (CABA)" y "pos x" / "pos y". En el primer caso tenemos geocodificacion plana, y en el segundo (las columnas de pos x y pos y) tenemos el sistema wgs84. Se verá cual tiene mayor cantidad de informacion, y si ocurre que una tiene informacion y la otra no. 

Primero imputaremos los datos con "sin dato". Notamos que cuando no hay informacion, en XY (CABA) se ve "point(. .)", mientras que en las otras dos columnas se rellenan con "." puntos. 

In [84]:
df['XY (CABA)'] = df['XY (CABA)'].replace('point (. .)', 'sin datos')
df['POS X'] = df['POS X'].replace('.', 'sin datos')
df['POS Y'] = df['POS Y'].replace('.', 'sin datos')

In [86]:
# Contar cuando 'XY (CABA)' tiene datos y 'pos x' y 'pos y' están ambos sin datos
conteo_datos_faltantes = df.apply(lambda x: ('sin datos' in x['POS X'] and 'sin datos' in x['POS Y'] and 'sin datos' not in x['XY (CABA)']), axis=1).value_counts()

In [87]:
conteo_datos_faltantes

False    695
Name: count, dtype: int64

In [88]:
# Contar cuando 'pos x' y 'pos y' tienen datos y 'XY (CABA)' está sin datos
conteo_datos_presentes = df.apply(lambda x: ('sin datos' in x['XY (CABA)'] and 'sin datos' not in x['POS X'] and 'sin datos' not in x['POS Y']), axis=1).value_counts()

In [53]:
conteo_datos_presentes

False    693
True       2
Name: count, dtype: int64

Como observamos, hay 2 casos en los que la columna XY (CABA) no tiene datos, pero la pos x y pos y si tienen. Se decide eliminar la primer columna y trabajar entonces con las columnas en geocodificacion wgs84

In [89]:
# Eliminar la columna 'XY (CABA)'
df = df.drop(columns=['XY (CABA)'])

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 695 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     695 non-null    object        
 1   N_VICTIMAS             695 non-null    int64         
 2   FECHA                  695 non-null    datetime64[ns]
 3   AAAA                   695 non-null    int64         
 4   MM                     695 non-null    int64         
 5   DD                     695 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    object        
 8   TIPO_DE_CALLE          695 non-null    object        
 9   CALLE                  695 non-null    object        
 10  ALTURA                 695 non-null    object        
 11  CRUCE                  695 non-null    object        
 12  DIRECCIÓN NORMALIZADA  695 non-null    object        
 13  COMUNA    

Finalmente tenemos nuestro set de datos limpio, sin nulos ni duplicados. Los valores faltantes fueron imputados y se han eliminado columnas innecesarias.

In [91]:
df.to_csv('../data/homicidios_clean.csv', index=False)